**Set environment**

In [1]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [6]:
cat  ${FD_RES}/results/region/region_screened_selected.bed | expand -t 12

chr11       4000000     6600000     chr11:4091884-6505900
chr11       32800000    35000000    chr11:32869701-34870100
chr11       61700000    62000000    chr11:61787329-61898348
chr12       54290000    54310000    chr12:54300766-54301042
chr12       110000000   113000000   chr12:110726151-112325737
chr16       0           1300000     chr16:10001-1173100
chr19       12880000    12900000    chr19:12887110-12887237
chr2        59000000    62000000    chr2:59553301-61553700
chr20       55500000    59000000    chr20:56391301-58391700
chr3        128475000   128512000   chr3:128487571-128487937
chr4        55000000    56500000    chr4:55237590-56068531
chr5        86000000    91600000    chr5:87611207-89911163
chr6        108000000   111000000   chr6:108840570-109828800
chr6        133000000   138000000   chr6:134253831-136927585
chr8        122000000   136000000   chr8:126735901-128736550
chrX        47000000    51000000    chrX:47785501-49880650


In [21]:
ASSAY=KS91_K562_ASTARRseq
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
ls ${FD_INP}/*WGS*bed* | wc -l
ls ${FD_INP}/*WGS*bed* | xargs -n 1 basename

10
KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep2.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep3.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep4.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep5.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Input.rep6.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep1.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep2.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep3.WGS.unstranded.bed.gz
KS91_K562_ASTARRseq.Output.rep4.WGS.unstranded.bed.gz


In [22]:
ASSAY=KS91_K562_ASTARRseq
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FP_INPS=($(ls ${FD_INP}/*WGS*bed*))
FP_INP=${FP_INPS[0]}
FN_INP=$(basename ${FP_INP})
FN_OUT=${FN_INP/WGS/SUBSET}

echo ${FP_INP}
echo
echo ${FN_INP}
echo
echo ${FN_OUT}

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz

KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz

KS91_K562_ASTARRseq.Input.rep1.SUBSET.unstranded.bed.gz


## RUN

In [28]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 8G \
    --output ${FD_LOG}/fragments_subset_astarr.%a.txt \
    --array 0-9 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=KS91_K562_ASTARRseq

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FP_INPS=($(ls ${FD_INP}/*WGS*bed*))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})

FD_OUT=${FD_RES}/results/${ASSAY}/fragment_count
FN_OUT=${FN_INP/WGS/SUBSET}
FP_OUT=${FD_OUT}/${FN_OUT}

FD_REG=${FD_RES}/results/region
FN_REG=region_screened_selected.bed
FP_REG=${FD_REG}/${FN_REG}

### show I/O file
echo "Input: " ${FP_INP}
echo
echo "show first few lines of the file"
zcat ${FP_INP} | head -5
echo
echo "Input: " ${FP_REG}
echo
echo "show lines of the file"
cat ${FP_REG}
echo

### annotation using intersect
bedtools intersect -a ${FP_INP} -b ${FP_REG} -wo | gzip -c > ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of the file:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29562356


## Check

In [29]:
cat ${FD_LOG}/fragments_subset_astarr.0.txt

Hostname:           x1-02-4.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-12-23+22:56:34

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Input.rep1.WGS.unstranded.bed.gz

show first few lines of the file
chr1	10001	10143	chr1_10001_10143	1	.
chr1	10002	10118	chr1_10002_10118	1	.
chr1	10002	10131	chr1_10002_10131	1	.
chr1	10002	10203	chr1_10002_10203	1	.
chr1	10002	10209	chr1_10002_10209	1	.

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/region_screened_selected.bed

show lines of the file
chr11	4000000	6600000	chr11:4091884-6505900
chr11	32800000	35000000	chr11:32869701-34870100
chr11	61700000	62000000	chr11:61787329-61898348
chr12	54290000	54310000	chr12:54300766-54301042
chr12	110000000	113000000	chr12:110726151-112325737
chr16	0	1300000	chr16:10001-1173100
chr19	12880000	12900000	chr19:12887110-12887237
chr2	59000000	62000000	chr2:59553301-61553700
chr20	55500000	590

In [30]:
cat ${FD_LOG}/fragments_subset_astarr.9.txt

Hostname:           x3-05-1.genome.duke.edu
Slurm Array Index:  9
Time Stamp:         03-12-23+22:56:34

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/KS91_K562_ASTARRseq/fragment_count/KS91_K562_ASTARRseq.Output.rep4.WGS.unstranded.bed.gz

show first few lines of the file
chr1	10033	10130	chr1_10033_10130	1	.
chr1	10057	10233	chr1_10057_10233	3	.
chr1	10104	10345	chr1_10104_10345	1	.
chr1	10116	10461	chr1_10116_10461	1	.
chr1	10147	10430	chr1_10147_10430	1	.

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/region_screened_selected.bed

show lines of the file
chr11	4000000	6600000	chr11:4091884-6505900
chr11	32800000	35000000	chr11:32869701-34870100
chr11	61700000	62000000	chr11:61787329-61898348
chr12	54290000	54310000	chr12:54300766-54301042
chr12	110000000	113000000	chr12:110726151-112325737
chr16	0	1300000	chr16:10001-1173100
chr19	12880000	12900000	chr19:12887110-12887237
chr2	59000000	62000000	chr2:59553301-61553700
chr20	55500000	59